In [1]:
import pandas as pd
import numpy as np

from urllib.request import urlopen
from urllib.parse import unquote
from bs4 import BeautifulSoup

import re
import time

import spacy
nlp = spacy.load("en_core_web_sm")


In [2]:
'''function to get all pages with artists'''

list_html=[]
def next_page(html):
    try:
        list_html.append(html)
        html_ukr=unquote(html, encoding='utf-8') #read cirilec
        #print(html_ukr)

        wiki='https://uk.wikipedia.org'
        html_open = urlopen(html)
        bs1 = BeautifulSoup(html_open, 'html.parser')
        link = bs1.find("div", {"id": "mw-pages"}).find('a',  string="наступна сторінка")['href']  
        html=wiki+link
        return next_page(html)
    except TypeError:
        print('We got all pages with artists\n\n')


html='https://uk.wikipedia.org/w/index.php?title=%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D1%81%D1%8C%D0%BA%D1%96_%D0%BA%D0%BE%D0%BC%D0%BF%D0%BE%D0%B7%D0%B8%D1%82%D0%BE%D1%80%D0%B8&pageuntil=%D0%94%D0%BE%D0%BD%D0%BD%D0%B8%D0%BA+%D0%9B%D0%B0%D1%80%D0%B8%D1%81%D0%B0+%D0%86%D0%B2%D0%B0%D0%BD%D1%96%D0%B2%D0%BD%D0%B0#mw-pages'

next_page(html)

for link in list_html:
    html_ukr=unquote(link, encoding='utf-8')
    print(html_ukr)

We got all pages with artists


https://uk.wikipedia.org/w/index.php?title=Категорія:Українські_композитори&pageuntil=Донник+Лариса+Іванівна#mw-pages
https://uk.wikipedia.org/w/index.php?title=Категорія:Українські_композитори&pagefrom=Донник+Лариса+Іванівна#mw-pages
https://uk.wikipedia.org/w/index.php?title=Категорія:Українські_композитори&pagefrom=Льонко+Євген+Олександрович#mw-pages
https://uk.wikipedia.org/w/index.php?title=Категорія:Українські_композитори&pagefrom=Сімович+Роман+Аполлонович#mw-pages


In [3]:
'''function to get the first sentence'''

list_first_sentence=[]
def first_sentence(htmls):
    html = urlopen(htmls)
    bs2 = BeautifulSoup(html, 'html.parser')
    name = bs2.find("span", {"class": "mw-page-title-main"}).text  #get the eng name of the article
    #print('name: ', name)
    text_first_sentance = bs2.find("div", {"id": "bodyContent"})\
                          .find("div", {"id": "mw-content-text"})\
                          .find("div", {"class": "mw-parser-output"})\
                          .find("p", class_=lambda x: x != 'mw-empty-elt').text
    #print('text_first_sentance: ', text_first_sentance)
    list_first_sentence.append(text_first_sentance)
    return name, text_first_sentance

fs=first_sentence('https://ar.wikipedia.org/wiki/%D8%AF%D9%85%D9%8A%D8%AA%D8%B1%D9%8A_%D9%84%D9%8A%D9%81%D9%8A%D8%AA%D8%B2%D9%83%D9%8A')
print('\n')
fs=first_sentence('https://en.wikipedia.org/wiki/Serhii_Brylov')
#print(fs)

In [6]:
'''function to find foreign version of artist'''

def jump_englis(htmls): 
    html_ukr=unquote(htmls, encoding='utf-8') #read cirilec
    html = urlopen(htmls)
    bs1 = BeautifulSoup(html, 'html.parser')

    #find name of artist
    ukr_name = bs1.find("h1", {"class": "firstHeading mw-first-heading"}).text

    #find list of all languages
    list_links=[]
    
    
    mydivs = bs1.find("nav", {"class": "vector-menu mw-portlet mw-portlet-lang vector-menu-portal portal"})\
            .find("ul", {"class": "vector-menu-content-list"})
    if re.match('<ul class="vector-menu-content-list"></ul>', str(mydivs)):
            len_links=0
            language=np.nan
            html_foreign=np.nan
            name_foreign=np.nan
            text_first_sentance=np.nan
            list_links.append({'Name UKR': ukr_name,'Number of languages': len_links,\
                'languages': language,'Name Foreign':  name_foreign, 'First sentance':  text_first_sentance,\
                'Link Foreign': html_foreign, 'Link UKR':  html_ukr})

    else: 
        links=mydivs.find_all('a', href = True)#re.compile('en\.wiki'))
        #if len(links)==0:
        #    status='not exist English'
        #else:
        len_links=len(links)
        for link in links:
            html_foreign=link['href']
            language=link['lang']
            #print(lang_eng, ':   ', html_eng)
            name_foreign, text_first_sentance=first_sentence(html_foreign)
            #print('\n ', name_foreign, text_first_sentance)
            list_links.append({'Name UKR': ukr_name,'Number of languages': len_links,\
                'languages': language,'Name Foreign':  name_foreign, 'First sentance':  text_first_sentance,\
                'Link Foreign': html_foreign, 'Link UKR':  html_ukr})
    df_one_person=pd.DataFrame(list_links)   
    return df_one_person 

one_person=(jump_englis('https://uk.wikipedia.org/wiki/%D0%91%D1%80%D0%B0%D1%82%D0%BA%D0%BE_%D0%86%D0%B2%D0%B0%D0%BD_%D0%92%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87'))
one_person

,Name UKR,Number of languages,languages,Name Foreign,First sentance,Link Foreign,Link UKR
0,Братко Іван Володимирович,0,NaN,NaN,NaN,NaN,https://uk.wikipedia.org/wiki/Братко_Іван_Воло...


In [7]:
'''function to find all links'''
start_time = time.time()
df=pd.DataFrame()

index_page=0
for html in list_html: # Don't forget to set the size [:2] when testing  !!!!
    #html='https://uk.wikipedia.org/w/index.php?title=%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D1%81%D1%8C%D0%BA%D1%96_%D1%85%D1%83%D0%B4%D0%BE%D0%B6%D0%BD%D0%B8%D0%BA%D0%B8&pagefrom=%D0%91%D1%80%D0%B0%D1%82%D0%BA%D0%BE+%D0%86%D0%B2%D0%B0%D0%BD+%D0%92%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87#mw-pages'
    #print(unquote(html, encoding='utf-8'))
    html_open = urlopen(html)
    bs1 = BeautifulSoup(html_open, 'html.parser')
    #name = bs1.find("div", {"id": "mw-pages"}).find("div", {"class": "mw-category-group"}).find_all('a', href=True)
    name = bs1.find("div", {"id": "mw-pages"}).find("div", {"class": "mw-content-ltr"}).find_all('a', href=True)
    index_link=0
    for link in name: # Don't forget to set the size [:5] when testing !!!!
        
        html='https://uk.wikipedia.org'+link['href']
        #print(html)
        list_df = jump_englis(html) 
        df=df.append(list_df) 

        #індексація щоб не нудно було очікувати
        index_link+=1
        if index_link % 50 == 0:
            print('Отримано {} посилань на сторінці {}'.format(index_link, index_page+1))
            
    index_page+=1
    print('\nСторінка {} готова\n'.format(index_page))
    
print("minute",  (time.time() - start_time)//60 )
df=df.reset_index(drop=True)
df

Отримано 50 посилань на сторінці 1
Отримано 100 посилань на сторінці 1
Отримано 150 посилань на сторінці 1
Отримано 200 посилань на сторінці 1

Сторінка 1 готова

Отримано 50 посилань на сторінці 2
Отримано 100 посилань на сторінці 2
Отримано 150 посилань на сторінці 2
Отримано 200 посилань на сторінці 2

Сторінка 2 готова

Отримано 50 посилань на сторінці 3
Отримано 100 посилань на сторінці 3
Отримано 150 посилань на сторінці 3
Отримано 200 посилань на сторінці 3

Сторінка 3 готова

Отримано 50 посилань на сторінці 4
Отримано 100 посилань на сторінці 4
Отримано 150 посилань на сторінці 4

Сторінка 4 готова

minute 26.0


,Name UKR,Number of languages,languages,Name Foreign,First sentance,Link Foreign,Link UKR
0,Абаза Аркадій Максимович,6,arz,اركادى ابازا,اركادى ابازا كان ملحن مزيكا من الامبراطوريه ال...,https://arz.wikipedia.org/wiki/%D8%A7%D8%B1%D9...,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...
1,Абаза Аркадій Максимович,6,cs,Arkadij Maximovič Abaza,Arkadij Maximovič Abaza (rusky Аркадий Максимо...,https://cs.wikipedia.org/wiki/Arkadij_Maximovi...,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...
2,Абаза Аркадій Максимович,6,de,Arkadi Maximowitsch Abasa,Arkadi Maximowitsch Abasa (russisch Аркадий Ма...,https://de.wikipedia.org/wiki/Arkadi_Maximowit...,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...
3,Абаза Аркадій Максимович,6,en,Arkady Abaza,Arkady Maksimovich Abaza (Абаза́ Арка́дий Макс...,https://en.wikipedia.org/wiki/Arkady_Abaza,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...
4,Абаза Аркадій Максимович,6,hy,Արկադի Աբազա,"Արկադի Աբազա (ռուս.՝ Абаза, Аркадий Максимович...",https://hy.wikipedia.org/wiki/%D4%B1%D6%80%D5%...,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...
...,...,...,...,...,...,...,...
2015,Яциневич Яків Михайлович,4,ru,"Яциневич, Яков Михайлович",Яков Михайлович Яциневич (8 ноября 1869 — 25 а...,https://ru.wikipedia.org/wiki/%D0%AF%D1%86%D0%...,https://uk.wikipedia.org/wiki/Яциневич_Яків_Ми...
2016,Ященко Леопольд Іванович,1,ru,"Ященко, Леопольд Иванович",Леопольд Иванович Я́щенко (1928—2016) — советс...,https://ru.wikipedia.org/wiki/%D0%AF%D1%89%D0%...,https://uk.wikipedia.org/wiki/Ященко_Леопольд_...
2017,Moisei,0,NaN,NaN,NaN,NaN,https://uk.wikipedia.org/wiki/Moisei
2018,Moon Shot,1,ru,Moon Shot,"Moon Shot (настоящее имя Сергей Мошляк, 6 октя...",https://ru.wikipedia.org/wiki/Moon_Shot,https://uk.wikipedia.org/wiki/Moon_Shot


In [8]:
df.iloc[3].tolist()

['Абаза Аркадій Максимович',
 6,
 'en',
 'Arkady Abaza',
 'Arkady Maksimovich Abaza (Абаза́ Арка́дий Макси́мович; 1843 in Sverdlikovo, Kursk Governorate – January 16\xa0[O.S. January 3]\xa01915, in Kursk) was a Russian composer, journalist and pianist. He studied in St. Petersburg with Alexander Dreyschock for fortepiano and singing in the class of Camille Everardi. Later he was an associate of Hans von Bulow. He taught and was director of a music school in Sumy (1877–81) and then in Kursk (1882–1915). Among his students, at Kursk, was Nikolai Roslavets.[1]\n',
 'https://en.wikipedia.org/wiki/Arkady_Abaza',
 'https://uk.wikipedia.org/wiki/Абаза_Аркадій_Максимович']

In [9]:
df_en=df[df['languages']=='en']
df_en['First sentance'].head(10).tolist()

['Arkady Maksimovich Abaza (Абаза́ Арка́дий Макси́мович; 1843 in Sverdlikovo, Kursk Governorate – January 16\xa0[O.S. January 3]\xa01915, in Kursk) was a Russian composer, journalist and pianist. He studied in St. Petersburg with Alexander Dreyschock for fortepiano and singing in the class of Camille Everardi. Later he was an associate of Hans von Bulow. He taught and was director of a music school in Sumy (1877–81) and then in Kursk (1882–1915). Among his students, at Kursk, was Nikolai Roslavets.[1]\n',
 'Anatoly Avdievsky (ukr. Авдієвський Анатолій Тимофійович, August 16, 1933 – March 24, 2016) was a Ukrainian choir conductor mostly known as the long-term manager of The Veryovka Ukrainian Folk Choir.\n',
 'Svitlana Azarova (Ukrainian: Світлана Азарова; born 9 January 1976) is a Ukrainian-Dutch composer of contemporary classical music, originally from the Ukrainian SSR.\n',
 'Gregory Alchevsky (Russian: Григо́рий Алексе́евич Алче́вский; 1866 in Kharkov, Russian Empire – 1920 in Mosco

In [10]:
'''function to get nationality'''

def get_natinality(text):
    text=re.sub(r'\([^)]*\)', '', text)
    list_nat=[] # список можливих національностей
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'NORP':
            nationality=ent.text
            get_way ='NORP'
            list_nat.append(nationality) 
    if len(list_nat)==0:
        for ent in doc.ents:
            if ent.label_ == 'GPE':
                nationality=ent.text
                get_way ='GPE'
                list_nat.append(nationality) 
    if len(list_nat)==0:
        get_way ='Error'
        list_nat.append(text)
    return list_nat, get_way
      
    
#To get the right part of DataFram  
df_nation=df[df['Number of languages']>=1]
df_nation = df_nation.drop(df_nation[df_nation['Name UKR'] == 'Список українських художників'].index)
#.reset_index(drop=True)
df_nation=df_nation[['languages','Name Foreign','First sentance']]
df_eng=df_nation[df_nation['languages']=='en']
df_eng['Nationality List'], df_eng['Get_way']=zip(*df_eng['First sentance'].apply(get_natinality))
df_eng


/var/folders/wj/pfkbghvd6r1cgqy3gcg46_8w0000gn/T/ipykernel_92111/3788612839.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['Nationality List'], df_eng['Get_way']=zip(*df_eng['First sentance'].apply(get_natinality))


,languages,Name Foreign,First sentance,Nationality List,Get_way
3,en,Arkady Abaza,Arkady Maksimovich Abaza (Абаза́ Арка́дий Макс...,[Russian],NORP
11,en,Anatoly Avdievsky,Anatoly Avdievsky (ukr. Авдієвський Анатолій Т...,[Ukrainian],NORP
17,en,Svitlana Azarova,Svitlana Azarova (Ukrainian: Світлана Азарова;...,"[Ukrainian, Ukrainian]",NORP
40,en,Gregory Alchevsky,Gregory Alchevsky (Russian: Григо́рий Алексе́е...,"[Ukrainian, Russian]",NORP
54,en,Mykola Arkas,Mykola Mykolayovych Arkas (born 7 January 1853...,[Ukrainian],NORP
...,...,...,...,...,...
1973,en,Alexander Shchetynsky,Alexander Shchetynsky (Shchetinsky) (Ukrainian...,"[Ukrainian, Ukrainian]",NORP
1982,en,Ludmila Yurina,Ludmila Yurina (born 16 January 1962) is a Ukr...,[Ukrainian],NORP
2002,en,Yaroslav Yaroslavenko,"Yaroslav Yaroslavenko (March 30, 1880 – June 2...","[Ukrainian, Ukrainian]",NORP
2008,en,Marceli Jasiński,Marceli Jasiński (Russian Марцелий Антонович Я...,[Polish],NORP


In [11]:
#pd.merge(df, df_nation, left_index=True, right_index=True) #inner join
df2=pd.concat([df, df_eng[['Nationality List', 'Get_way']]], axis=1)
df2

,Name UKR,Number of languages,languages,Name Foreign,First sentance,Link Foreign,Link UKR,Nationality List,Get_way
0,Абаза Аркадій Максимович,6,arz,اركادى ابازا,اركادى ابازا كان ملحن مزيكا من الامبراطوريه ال...,https://arz.wikipedia.org/wiki/%D8%A7%D8%B1%D9...,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...,NaN,NaN
1,Абаза Аркадій Максимович,6,cs,Arkadij Maximovič Abaza,Arkadij Maximovič Abaza (rusky Аркадий Максимо...,https://cs.wikipedia.org/wiki/Arkadij_Maximovi...,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...,NaN,NaN
2,Абаза Аркадій Максимович,6,de,Arkadi Maximowitsch Abasa,Arkadi Maximowitsch Abasa (russisch Аркадий Ма...,https://de.wikipedia.org/wiki/Arkadi_Maximowit...,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...,NaN,NaN
3,Абаза Аркадій Максимович,6,en,Arkady Abaza,Arkady Maksimovich Abaza (Абаза́ Арка́дий Макс...,https://en.wikipedia.org/wiki/Arkady_Abaza,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...,[Russian],NORP
4,Абаза Аркадій Максимович,6,hy,Արկադի Աբազա,"Արկադի Աբազա (ռուս.՝ Абаза, Аркадий Максимович...",https://hy.wikipedia.org/wiki/%D4%B1%D6%80%D5%...,https://uk.wikipedia.org/wiki/Абаза_Аркадій_Ма...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2015,Яциневич Яків Михайлович,4,ru,"Яциневич, Яков Михайлович",Яков Михайлович Яциневич (8 ноября 1869 — 25 а...,https://ru.wikipedia.org/wiki/%D0%AF%D1%86%D0%...,https://uk.wikipedia.org/wiki/Яциневич_Яків_Ми...,NaN,NaN
2016,Ященко Леопольд Іванович,1,ru,"Ященко, Леопольд Иванович",Леопольд Иванович Я́щенко (1928—2016) — советс...,https://ru.wikipedia.org/wiki/%D0%AF%D1%89%D0%...,https://uk.wikipedia.org/wiki/Ященко_Леопольд_...,NaN,NaN
2017,Moisei,0,NaN,NaN,NaN,NaN,https://uk.wikipedia.org/wiki/Moisei,NaN,NaN
2018,Moon Shot,1,ru,Moon Shot,"Moon Shot (настоящее имя Сергей Мошляк, 6 октя...",https://ru.wikipedia.org/wiki/Moon_Shot,https://uk.wikipedia.org/wiki/Moon_Shot,NaN,NaN


In [12]:
#df.to_excel("painter.xlsx", sheet_name='Eng')
df2.to_excel("Ukrainian_composers.xlsx",  sheet_name='Eng')